# Data Cleaning

In [37]:
import os
import pandas as pd
from ydata_profiling import ProfileReport

c:\Users\IvarCashinEriksson\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\IvarCashinEriksson\AppData\Local\Programs\Python\Python311\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Users\IvarCashinEriksson\AppData\Local\Programs\Python\Python311\Lib\site-packages\visions\backends\shared\nan_handling.py:50: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, b

### Read in data from file
The `trial_classes.xlsx` file has been enriched with additional multiclass labels identifying some characteristics of the trials.

In [38]:
data_directory = 'data'

data_dict = {}
for file_name in os.listdir(data_directory):
    file_path = os.path.join(data_directory, file_name)

    file_type = file_path.split('.')[-1]
    name = file_name.split('.')[0]
    if file_type == 'parquet':
        data_dict[name] = pd.read_parquet(file_path)
    elif file_type == 'xlsx':
        data_dict[name] = pd.read_excel(file_path)

---
### Clean data

#### Country

In [39]:
df = data_dict['country']

# Drop columns with more than 90 % nans
threshold = 0.9
df = df.dropna(thresh=len(df) * (1 - threshold), axis=1)

data_dict['country'] = df

#### Target

In [40]:
df = data_dict['target']

# Calculate target variable
efficiency = df['no_of_patients'] / df['enrolment_months']
df['efficiency'] = efficiency

# Drop rows with nan target value
df = df.dropna(subset=['efficiency'])

data_dict['target'] = df

#### Trial

In [41]:
df = data_dict['trial_classes']

# One hot encode trial phase
df_ohe = pd.get_dummies(df['trial_phase'], drop_first=True)
df_ohe = df_ohe.astype(int)
df['trial_phase_III'] = df_ohe

data_dict['trial_classes'] = df

#### Trial site

In [42]:
df = data_dict['trial_site']

df['site_start_date'] = pd.to_datetime(df['site_start_date'])
df['site_end_date'] = pd.to_datetime(df['site_end_date'])
df['trial_duration_days'] = (df['site_end_date'] - df['site_start_date']).dt.days

data_dict['trial_site'] = df

---
### Put it all together

In [43]:
df = pd.merge(data_dict['trial_site'], data_dict['country'], on='country_id', how='left')
df = pd.merge(df, data_dict['trial_classes'], on='trial_id', how='left')
df = pd.merge(df, data_dict['target'], on=['trial_id', 'site_id'], how='left')

---
### Make a smaller dataset for training

In [44]:
keep_cols = ['site_type', 'trial_duration_days', 'nn_region',
       'population', 'oecd_pharma_expenditure_per_capita',
       'oecd_medical_grads_per_1k', 'oecd_MDs_per_1k',
       'oecd_hospital_beds_per_1k', 'oecd_MR_units_per_1m',
       'oecd_pte_units_total', 'oecd_hypnotics_sedatives_per_1k',
       'oecd_antidepressants_per_1k', 'oecd_nervous_system_drugs_per_1k',
       'oecd_perc_pop_insured_by_gov_health',
       'oecd_perc_pop_insured_by_priv_health',
       'oecd_perc_pop_insured_by_priv_or_gov_health',
       'wb_diabetes_prevalence_perc_pop_age_20_to_79',
       'who_gho_ncd_paa_prev_insuff_physical_activity',
       'who_gho_ncd_paa_prev_insuff_physical_activity_male',
       'who_gho_ncd_paa_prev_insuff_physical_activity_female',
       'who_gho_mh_4_gov_exp_on_mental_perc_of_total_health',
       'who_gho_mh_12_suicide_rate_per100k',
       'who_gho_mh_16_beds_in_mental_hospitals_100k',
       'who_gho_mh_13_beds_for_mental_in_gen_hospitals_100k',
       'who_gho_gdo_q35_est_pop_prev_depression',
       'who_gho_whosis_000001_life_expec_at_birth_w',
       'who_gho_NCD_BMI_30A_obesity_prevalence_adults',
       'who_gho_ghed_oopsche_sha2011_oop_expenditure_pct_current_health_expenditure',
       'who_gho_ghed_che_pc_us_sha2011_curr_health_exp_per_capita_usd',
       'who_gho_ghed_che_pc_ppp_sha2011_curr_health_exp_per_capita_ppp',
       'who_gho_ghed_chegdp_sha2011_curr_health_exp_perc_gdp',
       'who_gho_sdgpm25_pm25_concentration_val',
       'who_gho_air_90_air_pollution_dalys_val',
       'who_gho_hwf_0025_prc_female_medical_doctors', 'minimum_age',
       'is_novo_nordisk_trial', 'is_top_20_sponsor', 'Heart Failure',
       'Cardiovascular Stability', 'High Consent Emphasis',
       'Technology-Enabled Monitoring', 'Weight Monitoring',
       'Medication and Treatment History', 'Kidney Function',
       'Cardiovascular Events', 'trial_phase_III', 'no_of_patients',
       'enrolment_months', 'efficiency']

df_training = df[keep_cols]

It would be nice to be able to clean out `NaN` values but after clearing rows with any `NaN`s 89 rows remain so this is not feasible. Similarly, if we do it on columns we only get 14 columns left. We need to do something smarter.

We elect to use a random forest as an initial machine learning approach as it is robust against missing data.

---
### Profile datasets

In [45]:
dfs_dict = {'full': df, 'training': df_training}
for key, df in dfs_dict.items():
    prof = ProfileReport(df)
    prof.to_file(output_file=f'profiles/{key}.html')

Summarize dataset:   0%|          | 0/63 [00:00<?, ?it/s, Describe variable:site_end_date]c:\Users\IvarCashinEriksson\AppData\Local\Programs\Python\Python311\Lib\site-packages\ydata_profiling\model\typeset_relations.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return not series.astype("datetime64").isna().all()
c:\Users\IvarCashinEriksson\AppData\Local\Programs\Python\Python311\Lib\site-packages\ydata_profiling\model\typeset_relations.py:88: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  return not series.astype("datetime64").isna().all()
Export report to file: 100%|██████████| 1/1 [00:00<00:00,  2.30it/s]
c:\Users\IvarCashinEriksson\AppData\Local\Programs\Python\Python311\Lib\site-packages\ydata_profili